In [6]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [8]:
IMAGE_SIZE = 200

In [5]:


labels = []
#read text file
with open('labels.txt') as f:
    labels = f.readlines()
    labels = [x.strip() for x in labels]


In [9]:
original_model = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape= (IMAGE_SIZE, IMAGE_SIZE, 3),
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

In [ ]:
# convert to tf lite
converter = tf.lite.TFLiteConverter.from_keras_model(original_model)
tflite_model = converter.convert()
#save the model
open("converted_model.tflite", "wb").write(tflite_model)
